In [1]:
from os import path
from collections import Counter
#import pycountry
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re # for removing numbers
from nltk.sentiment import SentimentIntensityAnalyzer
from gensim import models,corpora
%matplotlib inline

/Users/user/Desktop/epfl_semester5/data_analyis/anaconda_folder/anaconda/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
/Users/user/Desktop/epfl_semester5/data_analyis/anaconda_folder/anaconda/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
df = pd.read_csv(path.join('hillary-clinton-emails', 'emails.csv'))

In [15]:
df['RawText'][0]

'UNCLASSIFIED\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739545\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER.\nRELEASE IN FULL\nFrom: Sullivan, Jacob J <Sullivan11@state.gov>\nSent: Wednesday, September 12, 2012 10:16 AM\nTo:\nSubject: FW: Wow\nFrom: Brose, Christian (Armed Services) (mailto:Christian_Brose@armed-servic,essenate.govi\nSent: Wednesday, September 12, 2012 10:09 AM\nTo: Sullivan, Jacob J\nSubject: Wow\nWhat a wonderful, strong and moving statement by your boss. please tell her how much Sen. McCain appreciated it. Me\ntoo\nUNCLASSIFIED\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739545\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045247\n\x0c'

In [8]:
useful_data=df['RawText']
type(useful_data)

pandas.core.series.Series

In [16]:
useful_data=df['RawText']# useful_data=df['RawText']
#Delete words with numbers
without_num = [re.sub(r'\d+', '', t) for t in useful_data]

In [17]:
type(without_num)

list

In [18]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokennized_text=[tokenizer.tokenize(d) for d in without_num]

Delete words with less than 3 letters and also take care if it is from stopwords

In [20]:
#stopwords
sw = set(nltk.corpus.stopwords.words('english'))

In [21]:
tokennized_text=[[s for s in t if (len(s)>2 and s not in sw)] for t in tokennized_text]

In [44]:
#convert list of lists to one flattened list for data analysis
flattened = []
for t in tokennized_text:
    for w in t:
        flattened.append(w)

In [75]:
# now check words with highest frequency to see whether some of the words arent from email content 
from collections import Counter
counts = Counter(flattened)
print(counts)
#now delete words with highest frequencies

Counter({'State': 29830, 'Department': 28674, 'UNCLASSIFIED': 26910, 'Date': 26826, 'Case': 26564, 'Doc': 26539, 'From': 19269, 'Sent': 18847, 'Subject': 18421, 'state': 12797, 'The': 11733, 'gov': 11346, 'RELEASE': 7951, 'Message': 7314, 'Original': 7244, 'com': 6675, 'would': 5684, 'Huma': 5681, 'said': 5579, 'Cheryl': 5531, 'Abedin': 5191, 'clintonemail': 4995, 'Mills': 4763, 'PART': 4347, 'Secretary': 4005, 'Sullivan': 3908, 'call': 3824, 'FULL': 3588, 'Obama': 3548, 'Jacob': 3463, 'also': 3452, 'one': 3442, 'time': 3370, 'government': 3251, 'Clinton': 3088, 'President': 3022, 'people': 2969, 'new': 2863, 'This': 2813, 'know': 2783, 'HDR': 2764, 'STATE': 2646, 'work': 2579, 'like': 2532, 'But': 2525, 'get': 2504, 'could': 2480, 'United': 2428, 'AbedinH': 2382, 'American': 2379, 'security': 2342, 'http': 2285, 'two': 2218, 'May': 2165, 'Friday': 2129, 'And': 2091, 'Thursday': 2075, 'support': 2064, 'Haiti': 2062, 'see': 2044, 'need': 2042, 'Sunday': 2039, 'Monday': 2033, 'well': 202

In [76]:
# by checking the counts with corresponding words we can see that top 15 words arent useful for our purposes
highest_frequency=counts.most_common(15)

Delete words that appear in more than 90% of the documents. These words are coming from email headers that doesn't represent the email content

In [77]:
unuseful_words=[]
for i in range(0,len(highest_frequency)):
    unuseful_words.append(highest_frequency[i][0])

In [78]:
unuseful_words

['State',
 'Department',
 'UNCLASSIFIED',
 'Date',
 'Case',
 'Doc',
 'From',
 'Sent',
 'Subject',
 'state',
 'The',
 'gov',
 'RELEASE',
 'Message',
 'Original']

In [79]:
# Update tokenized text
tokennized_text_n=[[s for s in t if s not in unuseful_words] for t in tokennized_text]

In [80]:
dictionary = corpora.Dictionary(tokennized_text_n)
corpus = [dictionary.doc2bow(t) for t in tokennized_text_n]
lda=models.LdaModel(corpus, num_topics=5,id2word=dictionary)
lda.print_topics()

[(0,
  '0.010*"Secretary" + 0.007*"Office" + 0.003*"Room" + 0.003*"com" + 0.003*"would" + 0.003*"MEETING" + 0.003*"Clinton" + 0.003*"route" + 0.003*"PART" + 0.003*"ARRIVE"'),
 (1,
  '0.005*"said" + 0.005*"would" + 0.004*"Obama" + 0.003*"government" + 0.003*"Israel" + 0.003*"one" + 0.003*"American" + 0.003*"But" + 0.003*"people" + 0.003*"http"'),
 (2,
  '0.015*"com" + 0.012*"Huma" + 0.012*"clintonemail" + 0.011*"Abedin" + 0.009*"PART" + 0.008*"call" + 0.007*"HDR" + 0.005*"AbedinH" + 0.004*"hrod" + 0.004*"FULL"'),
 (3,
  '0.012*"Cheryl" + 0.011*"Mills" + 0.009*"Sullivan" + 0.009*"Jacob" + 0.005*"com" + 0.005*"FULL" + 0.005*"Huma" + 0.005*"clintonemail" + 0.005*"Secretary" + 0.004*"PART"'),
 (4,
  '0.006*"said" + 0.006*"Huma" + 0.005*"Abedin" + 0.005*"com" + 0.004*"AbedinH" + 0.004*"NEWS" + 0.004*"FULL" + 0.003*"call" + 0.003*"Reuters" + 0.003*"people"')]